In [1]:
import ast
import boto3
import pandas as pd

In [2]:
label_csv_david = 'D:/canopy_data/csvs/local_multi_label.csv'

label_df = pd.read_csv(label_csv_david, converters={1:ast.literal_eval})

label_df.head()

,filepaths,labels
0,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL]
1,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL]
2,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL]
3,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL]
4,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL]


In [3]:
type(label_df.loc[0, 'labels'])

list

In [4]:
value = label_df.loc[0, 'labels']

value

['ISL']

In [5]:
label_df['lengths'] = label_df['labels'].apply(lambda x:len(x))

label_df.head()

,filepaths,labels,lengths
0,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL],1
1,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL],1
2,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL],1
3,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL],1
4,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL],1


In [6]:
label_df['lengths'].value_counts()

1    34828
2     6558
3      356
4       29
Name: lengths, dtype: int64

## make dataframe of s3 filepaths

In [7]:
s3 = boto3.resource('s3')

bucket_name = 'canopy-production-ml'

pc_bucket = s3.Bucket(bucket_name)

print(pc_bucket)

s3.Bucket(name='canopy-production-ml')


In [8]:
train_chips = []

val_chips = []

train_uri = 'chips/cloudfree-merge-polygons/split/train/'

val_uri = 'chips/cloudfree-merge-polygons/split/val/'

for obj in pc_bucket.objects.all():
    #if train_uri in obj.key:
    #    train_chips.append(obj.key)
    if val_uri in obj.key:
        val_chips.append(obj.key)

In [9]:
len(val_chips)

6795

In [10]:
val_chips[0]

'chips/cloudfree-merge-polygons/split/val/1/1_1000_1000.tif'

In [11]:
df_val_s3 = pd.DataFrame(data={'s3_paths': val_chips})

df_val_s3.head()

,s3_paths
0,chips/cloudfree-merge-polygons/split/val/1/1_1...
1,chips/cloudfree-merge-polygons/split/val/1/1_1...
2,chips/cloudfree-merge-polygons/split/val/1/1_1...
3,chips/cloudfree-merge-polygons/split/val/1/1_1...
4,chips/cloudfree-merge-polygons/split/val/1/1_1...


## merge dataframes

In [12]:
test_1 = pd.DataFrame(data={'col1': [1,2,3], 'col2': [4,5,6]})

test_2 = pd.DataFrame(data={'col2': [5,6,4], 'col3': [8,9,10]})

test_1.merge(test_2, on='col2')

,col1,col2,col3
0,1,4,10
1,2,5,8
2,3,6,9


In [13]:
label_df.loc[0, 'filepaths']

'/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Chips/misha_polygons_cloudfreemerge/yes/ISL/100/59/59_800_2800.tif'

In [14]:
df_val_s3.loc[0, 's3_paths']

'chips/cloudfree-merge-polygons/split/val/1/1_1000_1000.tif'

In [15]:
def get_filename(path):
    return path.split('/')[-1]

label_df['filenames'] = label_df['filepaths'].apply(get_filename)
df_val_s3['filenames'] = df_val_s3['s3_paths'].apply(get_filename)

In [16]:
label_df.head()

,filepaths,labels,lengths,filenames
0,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL],1,59_800_2800.tif
1,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL],1,59_900_2800.tif
2,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL],1,59_900_2900.tif
3,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL],1,59_1000_2900.tif
4,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL],1,59_1000_3000.tif


In [17]:
df_val_s3.head()

,s3_paths,filenames
0,chips/cloudfree-merge-polygons/split/val/1/1_1...,1_1000_1000.tif
1,chips/cloudfree-merge-polygons/split/val/1/1_1...,1_1000_1100.tif
2,chips/cloudfree-merge-polygons/split/val/1/1_1...,1_1000_1200.tif
3,chips/cloudfree-merge-polygons/split/val/1/1_1...,1_1000_1300.tif
4,chips/cloudfree-merge-polygons/split/val/1/1_1...,1_1000_1400.tif


In [18]:
merge_df_val = df_val_s3.merge(label_df, on='filenames')

merge_df_val.head()

,s3_paths,filenames,filepaths,labels,lengths
0,chips/cloudfree-merge-polygons/split/val/1/1_1...,1_1000_1000.tif,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,"[ISL, Rainforest]",2
1,chips/cloudfree-merge-polygons/split/val/1/1_1...,1_1000_1100.tif,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[Rainforest],1
2,chips/cloudfree-merge-polygons/split/val/1/1_1...,1_1000_1200.tif,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[Rainforest],1
3,chips/cloudfree-merge-polygons/split/val/1/1_1...,1_1000_1300.tif,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[Rainforest],1
4,chips/cloudfree-merge-polygons/split/val/1/1_1...,1_1000_1400.tif,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[Rainforest],1


In [19]:
assert len(merge_df_val) == len(val_chips)

In [20]:
from sklearn.preprocessing import MultiLabelBinarizer


mlb = MultiLabelBinarizer()

val_ohe = mlb.fit_transform(merge_df_val['labels'])

val_ohe

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [22]:
mlb.classes_

array(['Habitation', 'ISL', 'Industrial_agriculture', 'Mining',
       'Rainforest', 'River', 'Roads', 'Savannah', 'Shifting_cultivation',
       'Water'], dtype=object)

In [21]:
list(val_ohe)[0]

array([0, 1, 0, 0, 1, 0, 0, 0, 0, 0])

In [23]:
len(val_ohe)

6795

In [26]:
with open('val.lst', 'w') as lst:
    for i in range(len(merge_df_val)):
        lst.write(str(i) + '\t')
        for label in list(val_ohe)[i]:
            lst.write(str(label) + '\t')
        lst.write(merge_df_val.loc[i, 's3_paths'])
        lst.write('\n')

## the function

In [27]:
from sklearn.preprocessing import MultiLabelBinarizer


def make_lst_file(label_df, chips, lst_path):
    s3_df = pd.DataFrame(data={'s3_paths': chips})
    
    def get_filename(path):
        return path.split('/')[-1]

    label_df['filenames'] = label_df['filepaths'].apply(get_filename)
    s3_df['filenames'] = df_val_s3['s3_paths'].apply(get_filename)
    
    merge_df = s3_df.merge(label_df, on='filenames')
    
    mlb = MultiLabelBinarizer()
    ohe = mlb.fit_transform(merge_df_val['labels'])

    with open(lst_path, 'w') as lst:
        for i in range(len(merge_df)):
            lst.write(str(i) + '\t')
            for label in list(ohe)[i]:
                lst.write(str(label) + '\t')
            lst.write(merge_df.loc[i, 's3_paths'])
            lst.write('\n')
            
    return mlb.classes_

In [28]:
make_lst_file(label_df, val_chips, 'val_2.lst')

array(['Habitation', 'ISL', 'Industrial_agriculture', 'Mining',
       'Rainforest', 'River', 'Roads', 'Savannah', 'Shifting_cultivation',
       'Water'], dtype=object)

## label json

In [1]:
import json


label_list = ['Habitation', 'ISL', 'Industrial_agriculture', 'Mining',
    'Rainforest', 'River', 'Roads', 'Savannah', 'Shifting_cultivation',
    'Water'
]

label_dict = {
    'label_names': {}
}

In [2]:
for i, label in enumerate(label_list, 1):
    label_dict['label_names'][i] = label

In [3]:
label_dict

{'label_names': {1: 'Habitation',
  2: 'ISL',
  3: 'Industrial_agriculture',
  4: 'Mining',
  5: 'Rainforest',
  6: 'River',
  7: 'Roads',
  8: 'Savannah',
  9: 'Shifting_cultivation',
  10: 'Water'}}

In [4]:
with open('labels.json', 'w') as json_file:
    json.dump(label_dict, json_file)